In [1]:
# NiN 块
import d2lzh as d2l
from mxnet import gluon, init,nd
from mxnet.gluon import nn

def nin_block(num_channels, kernel_size, strides, padding):
    blk = nn.Sequential()
    blk.add(nn.Conv2D(num_channels, kernel_size, strides, padding, activation='relu'),
            nn.Conv2D(num_channels, kernel_size=1, activation='relu'),
            nn.Conv2D(num_channels, kernel_size=1, activation='relu'))
    return blk

In [2]:
net = nn.Sequential()
net.add(nin_block(96, kernel_size=11, strides=4, padding=0),
        nn.MaxPool2D(pool_size=3, strides=2),
        nin_block(256, kernel_size=5, strides=1, padding=2),
        nn.MaxPool2D(pool_size=3, strides=2),
        nin_block(256, kernel_size=3, strides=1, padding=1),
        nn.MaxPool2D(pool_size=3, strides=2),
        # nin担任全链接角色  标签类别是10
        nin_block(10, kernel_size=3, strides=1, padding=1),
        # 全局平均池化层将窗口形状自动设置成输入的高和宽
        nn.GlobalAvgPool2D(),
        # 将四维的输出转换成二维的输出， 其形状为（批量大小， 10）
        nn.Flatten())

In [3]:
X = nd.random.uniform(shape=(1, 1, 224, 224))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape :\t ', X.shape)

sequential1 output shape :	  (1, 96, 54, 54)
pool0 output shape :	  (1, 96, 26, 26)
sequential2 output shape :	  (1, 256, 26, 26)
pool1 output shape :	  (1, 256, 12, 12)
sequential3 output shape :	  (1, 256, 12, 12)
pool2 output shape :	  (1, 256, 5, 5)
sequential4 output shape :	  (1, 10, 5, 5)
pool3 output shape :	  (1, 10, 1, 1)
flatten0 output shape :	  (1, 10)


In [4]:
lr, num_epochs, batch_size, ctx = 0.1, 5, 128, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on gpu(0)
epoch 1, loss 2.0153, train acc 0.245, test acc 0.533, time 126.2 sec
epoch 2, loss 0.9808, train acc 0.621, test acc 0.733, time 115.9 sec
epoch 3, loss 0.6315, train acc 0.761, test acc 0.807, time 116.4 sec
epoch 4, loss 0.5389, train acc 0.800, test acc 0.826, time 117.1 sec
epoch 5, loss 0.4715, train acc 0.826, test acc 0.851, time 117.1 sec
